In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.10.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.FileExtensions, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

# Load OpenAI key from dotnet User-Secrets and setup Semantic Kernel

In [ ]:
using System.IO;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.Extensions.Configuration;


var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile(Path.Combine(
        Environment.GetFolderPath(Environment.SpecialFolder.ApplicationData),
        "Microsoft",
        "UserSecrets",
        "2b6c897c-02c0-4c8e-91a8-cb61113a6acc", // comes from the project file QuokkaVec.Console.csproj
        "secrets.json")
    );

IConfigurationRoot configuration = builder.Build();

string openAIAPIKey = configuration["OpenAIAPIKey"];
if (string.IsNullOrWhiteSpace(openAIAPIKey))
{
	Console.WriteLine("OpenAIAPIKey is not set, use 'dotnet user-secrets init' and 'dotnet user-secrets set OpenAIAPIKey <your key>' to set the key, exiting...");
	return;
}

Console.WriteLine("OpenAIAPIKey is set");

#pragma warning disable SKEXP0010 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
#pragma warning disable SKEXP0001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var kerbelBuilder = Kernel.CreateBuilder();
kerbelBuilder.AddOpenAITextEmbeddingGeneration("text-embedding-3-small", openAIAPIKey);
var kernel = kerbelBuilder.Build();

var embeddingGenerationService = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

In [ ]:
#r "nuget: Qdrant.Client, 1.9.0"

In [ ]:
docker start quokkavec-qdrant

# Setup Qdrant vector DB, setup Collection and fill it with embeddings

In [ ]:
var pathToIndex = "c:/temp/quokkavec-test";

In [ ]:
using Qdrant.Client;
using Qdrant.Client.Grpc;

#pragma warning disable SKEXP0010 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
#pragma warning disable SKEXP0001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var collectionName = "test";
var qdrantClient = new QdrantClient("localhost");
if (!await qdrantClient.CollectionExistsAsync(collectionName))
{
	await qdrantClient.CreateCollectionAsync(collectionName, new VectorParams { Size = 1536, Distance = Distance.Cosine });
}

// index all files in the folder
var files = Directory.GetFiles(pathToIndex, "*", SearchOption.AllDirectories);
int i = 0;
foreach (var file in files)
{
	var text = await File.ReadAllTextAsync(file);
	// take only first 1k characters
	text = text.Substring(0, Math.Min(1000, text.Length));
	ReadOnlyMemory<float> embedding = await embeddingGenerationService.GenerateEmbeddingAsync(text);

	var points = new List<PointStruct>()
	{
		new PointStruct
		{
			Id = new PointId { Num = (ulong)i },
			Vectors = embedding.ToArray(),
			Payload = { { "fileName", file } }
		}
	};
	var updateResult = await qdrantClient.UpsertAsync(collectionName, points);
	Console.WriteLine($"File {file} indexed, result: {updateResult}");

	i++;
}

In [ ]:
string searchQuery = "Latest news";

In [ ]:
#pragma warning disable SKEXP0010 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
#pragma warning disable SKEXP0001 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

var queryVector = await embeddingGenerationService.GenerateEmbeddingAsync(searchQuery);
var searchResult = await qdrantClient.SearchAsync(collectionName, queryVector.ToArray(), limit: 2);

foreach (var result in searchResult)
{
	Console.WriteLine($"File: {result.Payload["fileName"]}, Score: {result.Score}");
}